![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)



<p><font size="5" color='grey'> <b> Chat & Memory </b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
!uv pip install --system -q langgraph langchain_openai
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()



# 1 | Intro: Warum braucht KI ein Gedächtnis?
---


<p><font color='black' size="5">
Zustandslosigkeit von LLMs
</font></p>

Large Language Models (LLMs) wie GPT sind von Natur aus **zustandslos** – sie verfügen über kein eingebautes Gedächtnis. Jede Anfrage wird isoliert verarbeitet, ohne Bezug zu vorherigen Interaktionen. Deshalb muss der Chatverlauf (Historie) bei jeder Anfrage neu übergeben werden.

```
Ohne Memory:
User: "Mein Name ist Max"
AI: "Hallo Max!"
User: "Wie heiße ich?"
AI: "Das habe ich nicht gespeichert." ❌
```


# 2 | Short-term Memory
---


Kurzzeit-Memory speichert den Gesprächsverlauf einer aktiven Sitzung (eines Threads) und ermöglicht kontextbezogene Antworten.

## 2.1 | ... mit Python-Liste

Um zu verstehen, warum LangGraph in vielen Fällen nötig ist, hier ein Beispiel mit *manueller* Verwaltung.

In [ ]:
# Importe
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# System-Prompt
system_prompt = "Du bist ein hilfreicher und humorvoller KI-Assistent."

# Prompt-Template mit Historie (MessagesPlaceholder nimmt die Historie entgegen)
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# LLM
model_name = "gpt-4o-mini"
temperature = 0
llm = ChatOpenAI(model=model_name, temperature=temperature)

# Parser
parser = StrOutputParser()

# Chain
chain = prompt | llm | parser

In [ ]:
# Chat-Funktion mit manueller Historien-Verwaltung
def chat_manually(system_prompt, chat_history, user_input):
    """Führt eine Chat-Interaktion mit manueller Historien-Verwaltung durch."""

    # Chain aufrufen (Historie wird im Prompt mitgeschickt)
    response = chain.invoke({
        'system_prompt': system_prompt,
        'chat_history': chat_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### 🧑‍🦱 Mensch: \n{user_input}")
    mprint(f"### 🤖 KI: \n{response}\n")

    # ⭐ WICHTIG: Memory (Liste) muss MANUELL nach JEDEM Call aktualisiert werden
    chat_history.extend([HumanMessage(content=user_input), AIMessage(content=response)])

    return chat_history

In [ ]:
# Historie initialisieren
chat_history = []

# Konversation
chat_manually(system_prompt, chat_history, "Mein Name ist Max")
chat_manually(system_prompt, chat_history, "Ich mag Python-Programmierung")
chat_manually(system_prompt, chat_history, "Weißt du noch, wie ich heiße und was ich mag?")

mprint("### 📝 Gespeicherte Nachrichten (Liste):\n---")
for msg in chat_history:
    mprint(f"  **{msg.type}**:   {msg.content}")

mprint("\n\n❌ Problem: Keine Session-Verwaltung und fehleranfälliges, manuelles Memory-Management.")

## 2.2 | ... mit LangGraph



**LangGraph** automatisiert das Session- und Memory-Management mit dem **Checkpointer**. Dieser speichert den gesamten Zustand (`MessagesState`) eines **Threads** und lädt ihn beim nächsten Aufruf automatisch wieder. Das manuelle Aktualisieren entfällt.

Der Workflow/Graph ist sehr einfach: **START** -> **Chat** (LLM-Aufruf) -> **END**. Die Magie passiert durch den **`MessagesState`** und den **`Checkpointer`**.



In [ ]:
# Importe
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

from IPython.display import Image, display

<p><font color='black' size="5">
Workflow
</font></p>

In [ ]:
# Workflow: Ruft das Modell auf (der eigentliche Chatbot)
def chat_node(state: MessagesState):
    """ Diese Funktion wird bei jedem Chat-Schritt aufgerufen. Der 'state' enthält automatisch alle bisherigen Nachrichten.  """

    # System-Prompt hinzufügen (vor der Historie)
    # NOTE: Der MessagesState reduziert die Nachrichten beim Hinzufügen automatisch.
    messages = [
        SystemMessage(content=system_prompt)
    ] + state["messages"]

    # LLM aufrufen
    response = llm.invoke(messages)

    # 🛑 WICHTIG: Die Rückgabe MUSS eine Liste von Nachrichten sein!
    # Der MessagesState Reducer fügt die Liste der Historie hinzu.
    return {"messages": [response]}

<p><font color='black' size="5">
Graph
</font></p>

In [ ]:
# Graph
workflow = StateGraph(state_schema=MessagesState)

# Knoten hinzufügen
workflow.add_node("chat", chat_node)

# Kanten definieren
workflow.add_edge(START, "chat")
workflow.add_edge("chat", END)

# Checkpointer (MemorySaver) verwaltet den Zustand pro Thread_ID
checkpointer = MemorySaver()

# Graph kompilieren
graph =  workflow.compile(checkpointer=checkpointer)
display(Image(graph.get_graph().draw_mermaid_png()))

<p><font color='black' size="5">
Ausführung
</font></p>

In [ ]:
# Ausführung
def chat(thread_id, user_input):
    """Chattet mit dem Bot in einem bestimmten Thread."""

    # Config enthält die Thread-ID (Session-ID)
    config = {"configurable": {"thread_id": thread_id}}

    # Input vorbereiten
    input_message = {"messages": [HumanMessage(content=user_input)]}

    # Graph aufrufen - Memory wird vom Checkpointer automatisch geladen/gespeichert!
    result = graph.invoke(input_message, config=config)

    # Ausgabe
    mprint(f"**🧑‍🦱 [Thread: {thread_id}] Mensch:** \n{user_input}")
    mprint(f"**🤖 [Thread: {thread_id}] KI:** \n{result['messages'][-1].content}\n")

    return result['messages'][-1].content


<p><font color='black' size="5">
Multi-User (Parallele Threads)
</font></p>

Jede **`thread_id`** erhält ihre **eigene, isolierte Historie**.

In [ ]:
# Thread 1: Max
chat("thread_max", "Ich mag Python-Programmierung!")

# Thread 2: Emma (komplett separate Konversation)
chat("thread_emma", "Ich interessiere mich für Machine Learning!")

# Zurück zu Thread 1 - Max's Memory bleibt erhalten!
chat("thread_max", "Was war nochmal mein Interesse?")

# Zurück zu Thread 2 - Emma's Memory bleibt erhalten!
chat("thread_emma", "An welchem Thema bin ich interessiert?")
print()

In [ ]:
def show_thread_history(thread_id):
    """Zeigt die komplette Historie eines Threads."""

    config = {"configurable": {"thread_id": thread_id}}
    state = graph.get_state(config)

    mprint(f"### 📝 Thread '{thread_id}' - {len(state.values['messages'])} Nachrichten:\n")
    mprint("---")

    for msg in state.values["messages"]:
        role = "🧑‍🦱" if msg.type == "human" else "🤖"
        mprint(f"{role} {msg.type.upper()}: {msg.content}")

# Beispiel
show_thread_history("thread_max")
show_thread_history("thread_emma")


# 3 | Memory Management
---

**Problem:** Lange Konversationen sprengen das **Kontextfenster** und werden teuer (Kosten, Latenz). Auch moderne, große Modelle leiden unter dem **"Lost in the middle"**-Problem, bei dem wichtige Informationen in der Mitte eines langen Prompt leicht ignoriert werden.

**Lösung:** Intelligentes Memory-Management.

<p><font color='black' size="5">
Funktionsstruktur mit Management
</font></p>

Hinzufügen eines **Pre-Processing-Schritts** vor dem Chat, um die Nachrichten zu trimmen oder zusammenzufassen.




## 3.1 | Trimming (Sliding Window)



**Strategie:** Behalte nur die letzten *n* Nachrichten. Alles ältere wird entfernt (oder nur der letzte Teil des Prompts wird genutzt). LangChain bietet hierfür eingebaute Utilities.

In [ ]:
# Import
from langchain_core.messages import trim_messages

In [ ]:
# Memory-Einstellungen
max_messages_before_trim = 10
messages_to_summarize = 8
recent_messages_to_keep = 2

<p><font color='black' size="5">
Workflow
</font></p>

In [ ]:
# Trimming: Nur die letzten n Token/Nachrichten behalten
def chat_node_with_trimming(state: MessagesState):
    """Ruft das Modell mit getrimmter Historie auf."""

    # System-Prompt
    system_msg = SystemMessage(content=system_prompt)

    # Nur die letzten N Nachrichten (hier: max_messages_before_trim)
    # 'token_counter=len' zählt die Anzahl der Nachrichten statt Token
    trimmed = trim_messages(
        state["messages"],
        max_tokens=max_messages_before_trim,
        strategy="last",
        token_counter=len, # ← Zählt Nachrichten statt Tokens
        include_system=False # System-Nachricht nicht in das Limit einbeziehen
    )

    messages = [system_msg] + trimmed
    response = llm.invoke(messages)

    return {"messages": [response]}

<p><font color='black' size="5">
Graph
</font></p>

In [ ]:
# Graph mit Trimming
workflow_trimmed = StateGraph(state_schema=MessagesState)
workflow_trimmed.add_node("chat", chat_node_with_trimming)
workflow_trimmed.add_edge(START, "chat")
workflow_trimmed.add_edge("chat", END)

graph_trimmed = workflow_trimmed.compile(checkpointer=MemorySaver())
display(Image(graph.get_graph().draw_mermaid_png()))

<p><font color='black' size="5">
Ausführung
</font></p>

In [ ]:
# Ausführung
def chat_with_trimming(thread_id, user_input):
    """Chattet mit dem Bot unter Verwendung der Summary-Strategie."""

    config = {"configurable": {"thread_id": thread_id}}
    # User-Input ist die NEUE Nachricht und wird vom Checkpointer zur Historie hinzugefügt.
    input_message = {"messages": [HumanMessage(content=user_input)]}

    result = graph_trimmed.invoke(input_message, config=config)

    mprint(f"### 🧑‍🦱 Mensch: \n{user_input}")
    mprint(f"### 🤖 KI: \n{result['messages'][-1].content}\n")

    return result['messages'][-1].content

In [ ]:
# Beispiel: Lange Konversation (mehr als max_messages_before_trim)
thread = "trimming_test"

# Erste Nachrichten (unterhalb des Limits)
chat_with_trimming(thread, "Mein Name ist Max")
chat_with_trimming(thread, "Ich wohne in Köln")
chat_with_trimming(thread, "Ich mag Python")
chat_with_trimming(thread, "Ich habe eine Katze namens Neo")

# Jetzt wird das Limit überschritten und die Zusammenfassung ausgelöst
for i in range(7):
    chat_with_trimming(thread, f"Dies ist Test-Nachricht Nummer {i+1} zur Füllung der Historie.")

# Nach der Zusammenfassung: Die KI sollte trotzdem wichtige Infos kennen
chat_with_trimming(thread, "Wie hieß meine Katze?")
print()

## 3.2 | Summarization (Zusammenfassung)



**Strategie:** Fasse alte Nachrichten zusammen. Die Zusammenfassung ersetzt dann die älteren Nachrichten in der Historie, wodurch Platz gespart wird.

In [ ]:
# Import
from langchain_core.prompts import ChatPromptTemplate

<p><font color='black' size="5">
Workflow
</font></p>

In [ ]:
# Zusammenfassungs-Prompt
summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "Fasse die folgende Konversation in 2-3 Sätzen zusammen. Behalte wichtige Fakten und Präferenzen."),
    ("human", "{conversation}")
])

def summarize_conversation(messages):
    """Erstellt eine Zusammenfassung der Nachrichten."""

    # Konversation als Text formatieren
    conversation_text = "\n".join([
        f"{msg.type}: {msg.content}" for msg in messages
    ])

    # Zusammenfassung erstellen
    summary_chain = summarize_prompt | llm | StrOutputParser()
    summary = summary_chain.invoke({"conversation": conversation_text})

    return summary

# Zusammenfassung erstellen
def chat_node_with_summary(state: MessagesState):
    """Nutzt Zusammenfassung statt alter Nachrichten, wenn die Historie zu lang wird."""

    all_messages = state["messages"]
    summary = "—"

    # Wenn die Historie zu lang ist: Zusammenfassen
    if len(all_messages) > max_messages_before_trim:
        mprint(f"\n⚠️ Historie zu lang ({len(all_messages)}). Fasse ältere Nachrichten zusammen.\n")

        # 1. Ältere Nachrichten zum Zusammenfassen auswählen
        to_summarize = all_messages[:messages_to_summarize]
        summary = summarize_conversation(to_summarize)

        # 2. Den Prompt mit der Zusammenfassung und nur den letzten Nachrichten füllen
        messages_for_prompt = [
            SystemMessage(content=f"{system_prompt}\n\nBisheriger Kontext (Zusammenfassung): {summary}"),
        ] + all_messages[-recent_messages_to_keep:]

    else:
        # Wenn Historie kurz genug: Gesamte Historie (inkl. System-Prompt) verwenden
        messages_for_prompt = [SystemMessage(content=system_prompt)] + all_messages

    # LLM aufrufen (antwortet auf die letzte User-Nachricht + Kontext)
    response = llm.invoke(messages_for_prompt)

    # Wichtig: Wir geben NUR die KI-Antwort zurück, LangGraph speichert die User-Nachricht und die KI-Antwort im State!
    return {"messages": [response]}

<p><font color='black' size="5">
Graph
</font></p>

In [ ]:
# Graph mit Summary-Funktion erstellen
workflow_summary = StateGraph(state_schema=MessagesState)
workflow_summary.add_node("chat", chat_node_with_summary)
workflow_summary.add_edge(START, "chat")
workflow_summary.add_edge("chat", END)

# Mit Memory kompilieren
graph_summary = workflow_summary.compile(checkpointer=MemorySaver())
display(Image(graph.get_graph().draw_mermaid_png()))

<p><font color='black' size="5">
Ausführung
</font></p>

In [ ]:
# Ausführung
def chat_with_summary(thread_id, user_input):
    """Chattet mit dem Bot unter Verwendung der Summary-Strategie."""

    config = {"configurable": {"thread_id": thread_id}}
    # User-Input ist die NEUE Nachricht und wird vom Checkpointer zur Historie hinzugefügt.
    input_message = {"messages": [HumanMessage(content=user_input)]}

    result = graph_summary.invoke(input_message, config=config)

    mprint(f"### 🧑‍🦱 Mensch: \n{user_input}")
    mprint(f"### 🤖 KI: \n{result['messages'][-1].content}\n")

    return result['messages'][-1].content

In [ ]:
# Beispiel: Lange Konversation (mehr als max_messages_before_trim)
thread = "summary_test"

# Erste Nachrichten (unterhalb des Limits)
chat_with_summary(thread, "Mein Name ist Max")
chat_with_summary(thread, "Ich wohne in Köln")
chat_with_summary(thread, "Ich mag Python")
chat_with_summary(thread, "Ich habe eine Katze namens Neo")

# Jetzt wird das Limit überschritten und die Zusammenfassung ausgelöst
for i in range(7):
    chat_with_summary(thread, f"Dies ist Test-Nachricht Nummer {i+1} zur Füllung der Historie.")

# Nach der Zusammenfassung: Die KI sollte trotzdem wichtige Infos kennen
chat_with_summary(thread, "Wie hieß meine Katze?")
print()


# 4 | Long-term Memory - Ausblick
---

**Problem:** Short-term Memory ist chronologisch und begrenzt. Es eignet sich nicht für das Speichern von großen, permanenten Wissensbasen oder Fakten.

**Lösung:** LangGraph **Store** (Long-term Memory).

Der Store speichert Fakten in **Namespaces** (typischerweise `user_id` oder `topic`) und ermöglicht die **semantische Suche** (Vektorsuche), wodurch nur relevante Fakten in den Prompt geladen werden (RAG-Prinzip).

<p><font color='black' size="5">
Die zwei Memory-Typen in LangGraph
</font></p>

Im LangChain/LangGraph-Umfeld wird das Gedächtnis klar getrennt, um Multi-User-Systeme zu ermöglichen:

| Aspekt | Short-term (Checkpointer) | Long-term (Store) |
|--------|---------------------------|-------------------|
| **Zweck** | Gesprächskontext pro Thread | Wissensbasis über Threads hinweg |
| **Scope** | **`thread_id`** (Session) | **`user_id`** / Namespace (Global) |
| **Speicher** | Liste von Nachrichten (chronologisch) | Fakten / Präferenzen (semantisch) |
| **Herausforderung** | Längenbegrenzung (Trimming) | Relevante Suche (RAG) |
| **Technologie** | `InMemorySaver`, `SqliteSaver` | `InMemoryStore` mit Index, `PostgresStore` |


In [ ]:
# Importe
from langgraph.store.memory import InMemoryStore
from langchain_core.runnables import RunnableConfig

<p><font color='black' size="5">
Pre-Processing: Datensammlung
</font></p>

In [ ]:
# Store mit OpenAI Embeddings (1536 Dimensionen für text-embedding-3-small)
# 🛑 WICHTIG: Der Index ermöglicht die semantische (Ähnlichkeits-)Suche!
store = InMemoryStore(index={
    "dims": 1536,
    "embed": "openai:text-embedding-3-small"
})

# Fakten zu User "Max" speichern (Namespace: user/max)
USER_ID_MAX = "max_42"
user_ns = ("user", USER_ID_MAX)

# Speichere Schlüssel-Wert-Paare im Store (könnten auch aus Summarization kommen)
store.put(user_ns, "proj_2020", {"text": "Max hat 2020 ein Chatbot-Projekt gemacht"})
store.put(user_ns, "lang_python", {"text": "Maxs Lieblingssprache ist Python"})
store.put(user_ns, "city_koeln", {"text": "Max wohnt in Köln"})

print(f"✅ Long-term Store mit {len(store.search(user_ns, limit=10))} Einträgen für User '{USER_ID_MAX}' erstellt")

In [ ]:
# Semantische Suche (nicht nur String-Matching!)
def retrieve_memory_semantic(query: str, user_id: str, k: int = 2):
    """Sucht relevante Erinnerungen per semantischer Vektorsuche im Store."""
    ns = ("user", user_id)
    # 🛑 Semantische Suche mit 'query' im optionalen zweiten Argument
    results = store.search(ns, query=query, limit=k)

    if not results:
        return "(Keine relevanten Erinnerungen gefunden)"

    return "\n".join([item.value["text"] for item in results])

# Test der Suche
print("🔍 Semantische Suche: 'Welche Projekte?'")
print(retrieve_memory_semantic("Welche Projekte hat er gemacht?", USER_ID_MAX))
print("\n🔍 Semantische Suche: 'Programmiersprachen' (trotzdem Treffer bei 'Lieblingssprache')")
print(retrieve_memory_semantic("Programmiersprachen", USER_ID_MAX))

<p><font color='black' size="5">
Workflow
</font></p>

In [ ]:
# Chat mit Store-Integration
def chat_node_with_store(state: MessagesState, config: RunnableConfig):
    """Nutzt Long-term Store (Wissensbasis) zusätzlich zur Short-term Historie."""

    # 🛑 Wichtig: user_id wird als Namespace für den Store genutzt
    user_id = config["configurable"]["user_id"]
    user_query = state["messages"][-1].content

    # 1. Semantische Suche in Store (RAG-Schritt)
    memories = retrieve_memory_semantic(user_query, user_id, k=2)

    # 2. Relevante Fakten als System-Kontext hinzufügen
    messages = [
        SystemMessage(content=f"{system_prompt}\n\nRelevante Informationen aus Wissensbasis:\n{memories}")
    ] + state["messages"]

    response = llm.invoke(messages)
    return {"messages": [response]}


<p><font color='black' size="5">
Graph
</font></p>

In [ ]:
# Graph mit Store erstellen
workflow_store = StateGraph(state_schema=MessagesState)
workflow_store.add_node("chat", chat_node_with_store)
workflow_store.add_edge(START, "chat")
workflow_store.add_edge("chat", END)

# Short-term Checkpointer (MemorySaver) bleibt erhalten
graph_store = workflow_store.compile(checkpointer=MemorySaver())
display(Image(graph.get_graph().draw_mermaid_png()))

<p><font color='black' size="5">
Ausführung
</font></p>

In [ ]:
def chat_with_store(thread_id: str, user_id: str, user_input: str):
    """Chattet mit dem Bot unter Verwendung von Store (semantische Suche)."""

    config = {
        "configurable": {
            "thread_id": thread_id, # Short-term (Historie)
            "user_id": user_id      # Long-term (Store-Namespace)
        }
    }
    input_message = {"messages": [HumanMessage(content=user_input)]}

    result = graph_store.invoke(input_message, config=config)

    print(f"🧑‍🦱 Mensch: {user_input}")
    print(f"🤖 KI: {result['messages'][-1].content}\n")

    return result['messages'][-1].content

# Beispiel: Die KI kann auf Store-Wissensbasis zugreifen (semantische Suche!)
chat_with_store("store_max_t1", USER_ID_MAX, "Wann hat Max sein Chatbot-Projekt gemacht?")
chat_with_store("store_max_t1", USER_ID_MAX, "Welche Programmiersprache mag Max?")

# A | Aufgaben
---


Die Aufgabenstellungen unten bieten Anregungen. Sie können aber auch gerne eigene Aufgaben verwenden.



<p><font color='black' size="5">
Aufgabe 1: Multi-User Chatbot mit LangGraph (Short-term)
</font></p>

**Schwierigkeit:** ⭐⭐

Erstellen Sie einen Chatbot unter Verwendung von **Abschnitt 2** (`graph`), der:
- Mindestens 3 verschiedene User-Threads verwaltet (z.B. "max_session", "emma_session", "ralf_session").
- Jeden User beim Namen kennt.
- Beim Thread-Wechsel den Kontext des jeweiligen Users korrekt behält (zeigen Sie dies, indem Sie zwischen zwei Threads hin- und herwechseln).


<p><font color='black' size="5">
Aufgabe 2: Memory mit Trimming-Strategie
</font></p>

**Schwierigkeit:** ⭐⭐⭐

Implementieren Sie einen Chatbot unter Verwendung von **Abschnitt 3.2** (`graph_summary`), der:
1. **`max_messages_before_trim`** auf einen kleinen Wert (z.B. 5) setzt.
2. Eine längere Konversation (z.B. 7 Schritte) durchführt, sodass die **Zusammenfassungslogik** ausgelöst wird (achten Sie auf den `⚠️` Hinweis).
3. Prüfen Sie mit `show_thread_history(thread_id)`, ob der Checkpointer immer noch **alle 7 Nachrichten** speichert, aber der Chat-Node **nur die relevanten Nachrichten** verwendet.

*(Hinweis: Der Checkpointer speichert immer die volle Historie; die Management-Strategie entscheidet, was dem LLM präsentiert wird.)*